In [ ]:
%pip install wandb
%pip install graphviz
%pip install torchviz
import wandb
wandb.login()#doesnt detect WANDB_NOTEBOOK_NAME on windows

In [ ]:
wandb.init(project="test-project", entity="simclr-doctoral-research")

In [ ]:
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 10,
  "batch_size": 512
}
image_size = 32
image_depth = 3

In [ ]:
import os
import wandb
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST,CIFAR10
from torchvision.utils import save_image

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), image_depth, image_size, image_size)
    return x


num_epochs = 100
batch_size = 512
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# dataset_train = MNIST('./data', transform=img_transform, download=True,train = True)
# dataset_test = MNIST('./data', transform=img_transform, download=True,train = False)
dataset_train = CIFAR10('./data', transform=img_transform, download=True,train = True)
dataset_test = CIFAR10('./data', transform=img_transform, download=True,train = False)

dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)



class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(image_size * image_size*image_depth, 256),
            nn.ReLU(True),
            nn.Linear(256, 128))
        self.decoder = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(True),
            nn.Linear(256,image_size * image_size*image_depth), 
            nn.Tanh())
    def forward(self, x,only_encode=False):
        if only_encode:
            return self.encoder(x)
        x = self.encoder(x)
        x = self.decoder(x)
        return x



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(str(device)+" is being used")
auto_model = AutoEncoder().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(auto_model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).to(device)
        # ===================forward=====================
        output = auto_model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))
    wandb.log({"loss": loss})

    wandb.watch(auto_model)
    #print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data[0]))

pic = to_img(output.cpu().data)
save_image(pic, './mlp_img/image_final.png')


In [ ]:
#test of the loss of the encoder on test data
test_data = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

for data in test_data:
    img, _ = data
    img = img.view(img.size(0), -1)
    img = Variable(img).to(device)
    output = auto_model(img)
    pic = to_img(output.cpu().data)
    save_image(pic, './mlp_img/autencoder.png')
#accuracy of the autoencoder comparing input and output
loss_sum = 0
for data in test_data:
    img, _ = data
    img = img.view(img.size(0), -1)
    img = Variable(img).to(device)
    output = auto_model(img)
    loss = criterion(output, img)
    loss_sum += loss.item()

print(loss_sum/len(test_data))


In [ ]:
from torchviz import make_dot
y = auto_model(img)
make_dot(y, params=dict(list(auto_model.named_parameters()))).render("torchviz", format="png")

Now the encoder is going to be frozen

In [ ]:
#freeze the encoder
# print(model.state_dict())

auto_model.encoder[0].weight.requires_grad = False
auto_model.encoder[0].bias.requires_grad = False
auto_model.encoder[2].weight.requires_grad = False
auto_model.encoder[2].bias.requires_grad = False

#show that it has been frozen
for name, param in auto_model.named_parameters():
    print(name, param.requires_grad)


In [ ]:
# #save the encoder (optionnal)
# os.makedirs('./saved_models', exist_ok=True)#the frost in not saved
# torch.save(auto_model.state_dict(), './saved_models/autoencoder.pth')

In [ ]:
from linear_classifier import LinearClassifier

In [ ]:
# #load the encoder (optionnal)
# model.load_state_dict(torch.load('./saved_models/autoencoder.pth'))
# #add a mlp to the encoder
# model.add_module('linear_classifier', linear_classifier())



In [ ]:
#joined model
class JoinedModel(nn.Module):
    def __init__(self):
        super(JoinedModel, self).__init__()
        self.encoder = auto_model.encoder
        self.classifier = LinearClassifier()
    def forward(self, x):
        x = self.encoder(x)
        x = self.classifier(x)
        return x
joined_model = JoinedModel().to(device)

In [ ]:
#we make sure they are frozen (we should remove the one above )
joined_model.encoder[0].weight.requires_grad = False
joined_model.encoder[0].bias.requires_grad = False
joined_model.encoder[2].weight.requires_grad = False
joined_model.encoder[2].bias.requires_grad = False

In [ ]:
#verify weight are frozen

for name, param in joined_model.named_parameters():
    print(name, param.requires_grad)
print(joined_model.parameters)


In [ ]:
#a function that calculates the accuracy of the model on the test set
def test_accuracy(model, test_loader):
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.view(images.size(0), -1)
            images = Variable(images).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        return 100 * correct / total

In [ ]:
#train the classifcation head of the model on the train data
# model = model.to(device)
import numpy as np
import matplotlib.pyplot as plt
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
test_accuracy_list = []
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, joined_model.parameters()), lr=learning_rate, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()
for epoch in range(num_epochs):
    for data in dataloader:
        img, label = data
        img = img.view(img.size(0), -1)
        img = Variable(img).to(device)
        label = Variable(label).to(device)
        # ===================forward=====================
        output = joined_model(img)#This should train the classifier
        loss = criterion(output, label)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
            .format(epoch + 1, num_epochs, loss.item()))
    wandb.log({"loss": loss})
    wandb.watch(joined_model)
    #calculate the accuracy of the model on the test set
    test_accuracy_list.append(test_accuracy(joined_model, test_loader))
    print("Test accuracy: {}".format(test_accuracy_list[-1]))
#plot the accuracy of the model over the epochs
#epochs in axis x
epochs = np.arange(1, num_epochs+1)
#accuracy in axis y
accuracy = test_accuracy_list
plt.plot(epochs, accuracy, label='accuracy')
plt.legend()
plt.show()

In [ ]:
#evaluate the loss one the test data
loss_sum = 0

for data in test_data:
    img, label = data
    img = img.view(img.size(0), -1)
    img = Variable(img).to(device)
    label = Variable(label).to(device)
    output = joined_model(img)
    loss = criterion(output, label)
    loss_sum += loss.item()


print(loss_sum/len(test_data))


In [ ]:
#

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# from fastai.vision.all import show_image
#show results of the model


for data in test_data:
    img, label = data
    img = img.view(img.size(0), -1)
    img = Variable(img).to(device)
    #label = Variable(label).to(device)
    output = joined_model(img)
    pic = to_img(img.cpu().data)
    # save_image(pic, './mlp_img/image_test.png')
    #show predicted label for the image
    #print(output.argmax(dim=1))
    #show the true label for the image
    #print(label)
    #show the image
    plt.imshow(np.transpose(img.view(img.size(0), image_depth, image_size, image_size).cpu().data[0], (1,2, 0)))
    #show the label corresponding to the image
    #print class name

    plt.title(dataset_test.classes[label.cpu().data[0]]+" predicted :"+ dataset_test.classes[output.argmax(dim=1)[0]])
    plt.show()
    plt.close()

    

In [ ]:
# evaluate model:
joined_model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for data in test_data:
        img, label = data
        img = img.view(img.size(0), -1)
        img = Variable(img).to(device)
        label = Variable(label).to(device)
        output = joined_model(img)
        _, predicted = torch.max(output.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
    print("accuracy: ", correct/total)

        

In [ ]:
#save the joined_model
torch.save(joined_model.state_dict(), './saved_models/joined_model.pth')
